### Model training v3
This notebook contains training of the fine-tunned LlaMa3 model for medical assistant for final project of the Qualentum AI Solutions Development bootcamp.
We have already trained the model using Llama3 and ruslanmv/ai-medical-chatbot dataset (5000 rows) and with a dataset containing 11800 rows of data about medicine supplies and their uses. Now the model will be trained with the final dataset with 467 rows containing medicine leaflets and their summaries which is the most relevant data for the project.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2/model.safetensors.index.json
/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2/model-00003-of-00004.safetensors
/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2/config.json
/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2/model-00001-of-00004.safetensors
/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2/README.md
/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2/tokenizer.json
/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2/tokenizer_config.json
/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2/model-00004-of-00004.safetensors
/kaggle/input/llama-3-8b-medi

In [2]:
%%capture
%pip install -U transformers 
%pip install -U datasets 
%pip install -U accelerate 
%pip install -U peft 
%pip install -U trl 
%pip install -U bitsandbytes 
%pip install -U wandb

Importing the necessary Python packages for loading the dataset, model, and tokenizer and fine-tuning.

In [3]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import (
    LoraConfig,
    PeftModel,
    prepare_model_for_kbit_training,
    get_peft_model,
)
import os, torch, wandb
from datasets import load_dataset
from trl import SFTTrainer, setup_chat_format

2024-07-30 18:25:17.631529: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-30 18:25:17.631642: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-30 18:25:17.746602: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
from huggingface_hub import login
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

hf_token = user_secrets.get_secret("huggingface_token")

login(token = hf_token, add_to_git_credential=True)

wb_token = user_secrets.get_secret("wandb")

wandb.login(key=wb_token)
run = wandb.init(
    project='Medical assistant.',
    job_type="training", 
    anonymous="allow"
)

Token is valid (permission: write).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' credential helper as default.

git config --global credential.helper store

Read https://git-scm.com/book/en/v2/Git-Tools-Credential-Storage for more details.
Token has not been saved to git credential helper.
Your token has been saved to /root/.cache/huggingface/token
Login successful


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: v-mcsimov (v-mcsimov-universidad-de-la-rioja). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.17.5
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240730_182531-rs0xlqwv
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run dry-breeze-26
wandb: ⭐️ View project at https://wandb.ai/v-mcsimov-universidad-de-la-rioja/Medical%20assistant.
wandb: 🚀 View run at https://wandb.ai/v-mcsimov-universidad-de-la-rioja/Medical%20assistant./runs/rs0xlqwv


Setting the base model, dataset, and new model variables.

In [5]:
base_model = "/kaggle/input/llama-3-8b-medical-assistant-v2/transformers/ver2/1/llama-3-8b-medical-assistant-v2"
dataset_name = "/kaggle/input/leaflets-with-answers/df_leaflets_with_answers.csv"
new_model = "llama-3-8b-medical-assistant-v3"

Setting the data type and attention implementation.

In [6]:
torch_dtype = torch.float16
attn_implementation = "eager"

Loading the model using 4-bit precision to reduce memory usage and speed up the fine-tuning process.

In [7]:
# QLoRA config
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch_dtype,
    bnb_4bit_use_double_quant=True,
)

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    attn_implementation=attn_implementation
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Load tokenizer.

In [8]:
tokenizer = AutoTokenizer.from_pretrained(base_model)
model, tokenizer = setup_chat_format(model, tokenizer)

Adding the adapter to the layer to improve the training time and make the process more memory-efficient.

In [9]:
# LoRA config
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['up_proj', 'down_proj', 'gate_proj', 'k_proj', 'q_proj', 'v_proj', 'o_proj']
)
model = get_peft_model(model, peft_config)

Model description function.

In [10]:
def describe_model(model: AutoModelForCausalLM):
    n_params = model.num_parameters()
    print(f" - The model has {n_params/1e6:,.1f} millions parameters")
    print(f" - Architecture: {model.config.architectures[0]}")
    try:
        ctx = model.config.max_position_embeddings
    except:
        ctx = model.config.seq_length
    print(f" - Maximum context: {ctx:,} tokens")
    print(f" - Parameters format: {model.config.torch_dtype}")
    print(f" - Vocabulary size: {model.config.vocab_size:,} tokens")

describe_model(model)

 - The model has 8,072.2 millions parameters
 - Architecture: LlamaForCausalLM
 - Maximum context: 8,192 tokens
 - Parameters format: torch.float16
 - Vocabulary size: 128,258 tokens


### Loading the dataset.

In [11]:
#Importing the dataset
dataset = pd.read_csv(dataset_name)

In [12]:
dataset

,Patient,Doctor
0,WHAT YOU SHOULD KNOW ABOUT PULMICORT (bu...,**Pulmicort Turbohaler (budesonide)**\n\n**Use...
1,INFORMATION FOR THE PATIENT (each FRUMIL ...,### Summary for Patients on FRUMIL FORTE\n\n**...
2,Taxotere 20 mg Docetaxel Please re...,**Medicine Name**: Taxotere (Docetaxel) 20 mg\...
3,"Demser (metirosine, MSD) PLEASE READ A...",**Medicine Name:** Demser (metirosine)\n\n**Us...
4,Patient Information Leaflet tablets f...,### Patient Information Summary for Imigran Ta...
...,...,...
462,Patient Information Leaflet on Amoxil ...,### Amoxil Paediatric Suspension - Patient Inf...
463,WHAT YOU SHOULD KNOW ABOUT'ZESTORETIC' 10 ...,### Important Information About Zestoretic 10\...
464,‘PALUDRINE’ Please read this leaf...,### Important Information about Paludrine\n\n*...
465,Antepsin Suspension (Sucralfate) Plea...,### Summary of Antepsin Suspension (Sucralfate...


Converting Pandas Dataframe into HuggingFace Dataset.

In [13]:
from datasets import Dataset

dataset = Dataset.from_pandas(dataset)

def format_chat_template(row):
    row_json = [{"role": "user", "content": row["Patient"]},
               {"role": "assistant", "content": row["Doctor"]}]
    row["text"] = tokenizer.apply_chat_template(row_json, tokenize=False)
    return row

dataset = dataset.map(
    format_chat_template,
    num_proc=4,
)

dataset['text'][42]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=4):   0%|          | 0/467 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


"<|im_start|>user\n        What you should know about Distalgesic tabletsDistalgesic    Please read this leaflet carefully before you start to take your medicine.  It does not contain all the information about your medicine that you may need to know so if you have any questions, ask your doctor or pharmacist. This leaflet only applies to Distalgesic tablets.    What is in your medicine?   Your medicine is called Distalgesic, which is a brand of co-proxamol.  Its active ingredients are dextropropoxyphene hydrochloride and paracetamol.  Each white, pillow-shaped tablet contains 32.5mg dextropropoxyphene hydrochloride and 325mg paracetamol.  They are labelled DG.  The tablets also contain the inactive ingredients maize starch and magnesium stearate, and are coated with hydroxypropyl methyl cellulose 15, glycerine and titanium dioxide.  They come in blister packs of 100.   Distalgesic is one of a group of medicines called analgesics that help to ease pain.   Distalgesic is made by Eli Lill

Splitting the dataset into a training and validation set.

In [14]:
dataset = dataset.train_test_split(test_size=0.1)

In [15]:
dataset

DatasetDict({
    train: Dataset({
        features: ['Patient', 'Doctor', 'text'],
        num_rows: 420
    })
    test: Dataset({
        features: ['Patient', 'Doctor', 'text'],
        num_rows: 47
    })
})

### Complaining and training the model.
Setting the model hyperparameters so that we can run it on the Kaggle.
Fine-tuning the model for one epoch and logging the metrics using the Weights and Biases.

In [16]:
training_arguments = TrainingArguments(
    output_dir=new_model,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    optim="paged_adamw_32bit",
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=0.2,
    logging_steps=1,
    warmup_steps=10,
    logging_strategy="steps",
    learning_rate=2e-4,
    fp16=False,
    bf16=False,
    group_by_length=True,
    report_to="wandb"
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Setting up a supervised fine-tuning (SFT) trainer and provide a train and evaluation dataset, LoRA configuration, training argument, tokenizer, and model. We are keeping the max_seq_length to 512 to avoid exceeding GPU memory during training.

In [17]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    peft_config=peft_config,
    max_seq_length=512,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': max_seq_length, dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:280: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:318: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will over

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

Map:   0%|          | 0/47 [00:00<?, ? examples/s]

In [18]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
126,0.974600,1.277874
252,0.612200,1.165228
378,0.750500,1.088164
504,0.610900,1.147485
630,0.354600,1.137054


We detected that you are passing `past_key_values` as a tuple and this is deprecated and will be removed in v4.43. Please use an appropriate `Cache` class (https://huggingface.co/docs/transformers/v4.41.3/en/internal/generation_utils#transformers.Cache)


TrainOutput(global_step=630, training_loss=0.9315947738904802, metrics={'train_runtime': 3864.4029, 'train_samples_per_second': 0.326, 'train_steps_per_second': 0.163, 'total_flos': 2.913577099001856e+16, 'train_loss': 0.9315947738904802, 'epoch': 3.0})

#### Model evaluation.

The Weights & Biases generates the run history and summary.

In [19]:
wandb.finish()
model.config.use_cache = True

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:               eval/loss █▄▁▃▃
wandb:            eval/runtime ▅▆▇█▁
wandb: eval/samples_per_second ███▁█
wandb:   eval/steps_per_second ███▁█
wandb:             train/epoch ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:       train/global_step ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
wandb:         train/grad_norm ▂▂▂▂▁▂▁▁▂▂▂▂▂▁▂▃▃▃▃▄▃▃▂▄▃▂▃▄▃▄▃▅▄█▄▄▄▃▂▆
wandb:     train/learning_rate ▇███▇▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
wandb:              train/loss ██▇▆▆▆▇▆▅▆▆▅▄▅▅▅▃▃▄▄▄▅▅▅▅▂▄▂▂▂▂▃▃▄▃▁▂▁▁▃
wandb: 
wandb: Run summary:
wandb:                eval/loss 1.13705
wandb:             eval/runtime 64.5609
wandb:  eval/samples_per_second 0.728
wandb:    eval/steps_per_second 0.728
wandb:               total_flos 2.913577099001856e+16
wandb:              train/epoch 3.0
wandb:        train/global_step 630
wandb:          train/grad_norm 1.64639
wandb:      train/learning_

Evaluating the model on a sample patient query to check if it’s properly fine-tuned.

In [20]:
messages = [
    {
        "role": "user",
        "content": """
        Hello Doctor, please provide me with a brief summary of this medicine leaflet, 3-6 side effects, 
        important information, and recommendations in 250 words maximum:
        What you should know about Distalgesic tabletsDistalgesic    Please read this leaflet carefully before you start to take your medicine.  It does not contain all the information about your medicine that you may need to know so if you have any questions, ask your doctor or pharmacist. This leaflet only applies to Distalgesic tablets.    What is in your medicine?   Your medicine is called Distalgesic, which is a brand of co-proxamol.  Its active ingredients are dextropropoxyphene hydrochloride and paracetamol.  Each white, pillow-shaped tablet contains 32.5mg dextropropoxyphene hydrochloride and 325mg paracetamol.  They are labelled DG.  The tablets also contain the inactive ingredients maize starch and magnesium stearate, and are coated with hydroxypropyl methyl cellulose 15, glycerine and titanium dioxide.  They come in blister packs of 100.   Distalgesic is one of a group of medicines called analgesics that help to ease pain.   Distalgesic is made by Eli Lilly and Company Limited, Kingsclere Road, Basingstoke, Hampshire, RG21 6XA.  The product licence is held by Dista Products Limited, Dextra Court, Chapel Hill, Basingstoke, Hampshire RG21 5SY     Why Distalgesic?   Distalgesic is given to help reduce mild to moderate pain.     Before taking your medicine    Make sure it is sate for you to take Distalgesic. It you answer YES to any of the following questions or you are not sure about the answer, tell your doctor or pharmacist.      Have you ever been allergic to this or any other pain-killing medicine?       Do you suffer from depression or any other psychiatric illness?     Have you ever been an alcoholic or drug addict?     Are you taking any other medicine, especially 'blood thinner' medicine (such as warfarin), antidepressants or drugs for epilepsy?     Are you pregnant or could you be?     Are you breast-feeding?       Have you ever had problems with your liver or kidneys?      Do not take more Distalgesic than your doctor tells you to  .    Do not drink alcohol while taking Distalgesic.    It can be very dangerous.  Discuss this with your doctor if you have any questions.   If you take drugs such as antihistamines, tranquillisers or sleeping tablets when you are taking Distalgesic, you may become extremely sleepy.   Distalgesic on its own makes some people very sleepy, especially when they first start taking it.   Do not drive or use dangerous machinery if you feel sleepy.   Do not give Distalgesic to children.      Please read the back of this leaflet          What you should know about Distalgesic tablets - continued         Taking your medicine     Follow your doctor’s instructions.  Check the label to see how many tablets to take and how often to take them.      Usual dose: take 2 tablets three or four times a day.       Do not take more than 8 tablets in 24 hours.    If you are elderly, you may need to take fewer.  If you are not sure how many tablets to take, ask your doctor or pharmacist     Swallow the tablets with a drink of water.     Do not take more tablets than your doctor tells you to.  You can become addicted to these tablets if you take too many for a long time.     If you miss a dose, take another dose as soon as you can.  Then go on as before.  Never double up on the next dose to make up for the one missed.      OVERDOSE: If you, or anyone else, ever take too many Distalgesic tablets, dial 999 for an ambulance straight away.  Then phone your doctor.    An overdose can damage your liver or cause coma, convulsions or death.       While taking your medicine     Distalgesic tablets may cause side-effects.  These include feeling dizzy or tired, feeling sick and vomiting.  They will often go away if you lie down.  Tell your doctor if they are troublesome.     Very occasionally, Distalgesic may give you a rash, headaches or stomach ache.  It might also make you constipated or moody, or give you problems with your sight, hallucinations or jaundice.  If you have these or any other ill-effects that are troublesome, tell your doctor.       How to store your medicine     Do not use Distalgesic after the 'Use Before' date.     Keep your tablets at room temperature, in a dry, safe place where children cannot see or reach them.  Your tablets could harm them.     When your treatment is over, we suggest that you take any tablets you have left back to the pharmacist who will dispose of them safely.     REMEMBER: This medicine is for you.  Only a doctor can prescribe it.  Never give it to others.  It may harm them, even if their symptoms are the same as yours.   'Distalgesic' is a registered trademark of Dista Products Limited.   Date of Leaflet Preparation: November 1995 © Dista Products Limited, 1995.
        """
    }
]

prompt = tokenizer.apply_chat_template(messages, tokenize=False, 
                                       add_generation_prompt=True)

inputs = tokenizer(prompt, return_tensors='pt', padding=True, 
                   truncation=True).to("cuda")

outputs = model.generate(**inputs, max_new_tokens=350, 
                         num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(text.split("assistant")[1])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



**Medication Name:** Distalgesic

**Uses:**   Distalgesic is given to help reduce mild to moderate pain.

**Important Side Effects:**
1. Dizziness
2. Tiredness
3. Feeling sick and vomiting
4. Hallucinations
5. Jaundice
6. Rash
7. Headaches
8. Stomach ache
9. Constipation
10. Mood changes

**Recommendations:**
1. Do not take more than your doctor recommends.
2. Do not drink alcohol while taking Distalgesic. It can be very dangerous.
3. Do not take drugs such as antihistamines, tranquillisers or sleeping tablets when taking Distalgesic. It can be very dangerous.
4. Do not drive or use dangerous machinery if you feel sleepy.
5. Do not give Distalgesic to children. It can be dangerous for them.
6. If you are pregnant or breast-feeding, tell your doctor before taking Distalgesic.
7. If you take drugs such as antihistamines, tranquillisers or sleeping tablets when taking Distalgesic, you may become extremely sleepy. Do not drive or use dangerous machinery if you feel sleepy.
8. Do not take 

#### Saving the model file

In [21]:
trainer.model.save_pretrained(new_model)
trainer.model.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/McSimoff/llama-3-8b-medical-assistant-v3/commit/52223b0a8b802e8b6a12d6b2fd6872367735bc78', commit_message='Upload model', commit_description='', oid='52223b0a8b802e8b6a12d6b2fd6872367735bc78', pr_url=None, pr_revision=None, pr_num=None)